In [ ]:
pip install openai

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('TradeData_6_15_2025_16_41_2.csv', encoding='latin1')

In [1]:
import sqlite3

In [7]:
# Create/connect to SQLite DB (creates if not exists)
conn = sqlite3.connect("bursa_palmai_database.db")  

# Write DataFrame to SQLite (creates a table named 'mth_prod_data')
df.to_sql(
    name="test_gt",   # Table name
    con=conn,            # SQLite connection
    if_exists="replace", # Overwrite if table exists
    index=False          # Don't write DataFrame index
)

# Confirm
print("Table successfully created in SQLite!")

Table successfully created in SQLite!


In [13]:
# 1. Insert a single row using sqlite3
def insert_single_row(month, raw_mat, volume, company):
    try:
        cursor = conn.cursor()
        query = """
        INSERT INTO company_mthly_prod (month, raw_mat, volume, company)
        VALUES (?, ?, ?, ?)
        """
        cursor.execute(query, (month, raw_mat, volume, company))
        conn.commit()
        print(f"Inserted row: {month}, {raw_mat}, {volume}, {company}")
    except sqlite3.Error as e:
        print(f"Error inserting single row: {e}")

# 2. Insert multiple rows using sqlite3.executemany
def insert_multiple_rows(rows):
    try:
        cursor = conn.cursor()
        query = """
        INSERT INTO company_mthly_prod (month, raw_mat, volume, company)
        VALUES (?, ?, ?, ?)
        """
        cursor.executemany(query, rows)
        conn.commit()
        print(f"Inserted {cursor.rowcount} rows")
    except sqlite3.Error as e:
        print(f"Error inserting multiple rows: {e}")

# 3. Insert rows using pandas DataFrame
def insert_df_rows(df):
    try:
        df.to_sql("company_mthly_prod", conn, if_exists="append", index=False)
        print(f"Inserted {len(df)} rows via DataFrame")
    except Exception as e:
        print(f"Error inserting DataFrame: {e}")

In [36]:
conn = sqlite3.connect("bursa_palmai_database.db")  
query_result = pd.read_sql("SELECT * FROM company_mthly_prod where month = '2025-04'", conn)
print(query_result)

      month                                  raw_mat  volume company
0   2025-04                      Fresh Fruit Bunches  512137     KLK
1   2025-04                           Crude Palm Oil  126337     KLK
2   2025-04                              Palm Kernel   23931     KLK
3   2025-04                                   Rubber  206475     KLK
4   2025-04                      Fresh Fruit Bunches  764978     SDG
5   2025-04                           Crude Palm Oil  188952     SDG
6   2025-04                              Palm Kernel   42474     SDG
7   2025-04     Fresh Fruit Bunches (Own Production)  256547     IOI
8   2025-04  Fresh Fruit Bunches (Outside Purchases)   11682     IOI
9   2025-04                           Crude Palm Oil   55647     IOI
10  2025-04                              Palm Kernel    9834     IOI
11  2025-04                                   Rubber   23510     IOI
12  2025-04                      Fresh Fruit Bunches  374230     FGV
13  2025-04                       

In [35]:
# Multiple rows
rows = [
    ("2025-04", "Fresh Fruit Bunches", 374230, "FGV"),
    ("2025-04", "Crude Palm Oil", 245258, "FGV"),
    ("2025-04", "Palm Kernel", 59242, "FGV"),
    ("2025-04", "Rubber", 41898, "FGV")
]
insert_multiple_rows(rows)

Inserted 4 rows


In [34]:
def delete_rows(month, company):
    try:
        cursor = conn.cursor()
        query = """
        DELETE FROM company_mthly_prod
        WHERE month = ? AND company = ?
        """
        cursor.execute(query, (month, company))
        conn.commit()
        deleted_count = cursor.rowcount
        print(f"Deleted {deleted_count} rows where month = '{month}' and company = '{company}'")
        return deleted_count
    except sqlite3.Error as e:
        print(f"Error deleting rows: {e}")
        return 0
    finally:
        cursor.close()

# Execute deletion
delete_rows("2025-04", "FGV")

Deleted 8 rows where month = '2025-04' and company = 'FGV'


8

In [14]:
# Multiple rows
rows = [
    ("2025-04", "Fresh Fruit Bunches (Own Production)", 256547, "IOI"),
    ("2025-04", "Fresh Fruit Bunches (Outside Purchases)", 11682, "IOI"),
    ("2025-04", "Crude Palm Oil", 55647, "IOI"),
    ("2025-04", "Palm Kernel", 9834, "IOI"),
    ("2025-04", "Rubber", 23510, "IOI"),
    ("2025-04", "Fresh Fruit Bunches", 512137, "KLK"),
    ("2025-04", "Crude Palm Oil", 126337, "KLK"),
    ("2025-04", "Palm Kernel", 23931, "KLK"),
    ("2025-04", "Rubber", 206475, "KLK"),
    ("2025-05", "Fresh Fruit Bunches", 479118, "KLK"),
    ("2025-05", "Crude Palm Oil", 119357, "KLK"),
    ("2025-05", "Palm Kernel", 22567, "KLK"),
    ("2025-05", "Rubber", 286556, "KLK"),
    ("2025-04", "Fresh Fruit Bunches", 374230, "FGV"),
    ("2025-04", "Crude Palm Oil", 245258, "FGV"),
    ("2025-04", "Palm Kernel", 59242, "FGV"),
    ("2025-04", "Rubber", 41898, "FGV"),
    ("2025-04", "Fresh Fruit Bunches", 374230, "FGV"),
    ("2025-04", "Crude Palm Oil", 245258, "FGV"),
    ("2025-04", "Palm Kernel", 59242, "FGV"),
    ("2025-04", "Rubber", 41898, "FGV"),
    ("2025-05", "Fresh Fruit Bunches", 380740, "FGV"),
    ("2025-05", "Crude Palm Oil", 263031, "FGV"),
    ("2025-05", "Palm Kernel", 61756, "FGV"),
    ("2025-05", "Rubber", 87956, "FGV"),
    ("2025-05", "Coconut", 8680, "FGV"),
    ("2025-05", "Fresh Fruit Bunches", 794246, "SDG"),
    ("2025-05", "Crude Palm Oil", 200629, "SDG"),
    ("2025-05", "Palm Kernel", 44534, "SDG"),
    ("2025-04", "Fresh Fruit Bunches", 764978, "SDG"),
    ("2025-04", "Crude Palm Oil", 188952, "SDG"),
    ("2025-04", "Palm Kernel", 42474, "SDG")
]
insert_multiple_rows(rows)

Inserted 32 rows


In [ ]:
# Example usage
# Single row
insert_single_row("2025-03", "PalmCo", 5000.0)



# DataFrame insertion
new_data = pd.DataFrame([
    {"month": "2025-05", "company_name": "GreenPalm", "production_tonnes": 7000.0},
    {"month": "2025-05", "company_name": "OilCorp", "production_tonnes": 6500.0}
])
insert_df_rows(new_data)



In [33]:
import sqlite3

def list_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Query SQLite's internal schema table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    conn.close()
    return [table[0] for table in tables]

# Usage
tables = list_tables("bursa_palmai_database.db")
print("Tables in database:", tables)

Tables in database: ['company_mthly_prod', 'company_ext_rate', 'company_plt_area', 'mpob_stats', 'universal_mill_list']


In [34]:
def get_table_schema(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    schema = cursor.fetchall()
    conn.close()
    return schema

# Usage
print(get_table_schema("bursa_palmai_database.db", "company_plt_area"))

[(0, 'Company', 'TEXT', 0, None, 0), (1, 'Category', 'TEXT', 0, None, 0), (2, 'Year', 'INTEGER', 0, None, 0), (3, 'Value', 'INTEGER', 0, None, 0)]


#Concession weather forecast

In [2]:
pd.set_option('display.max_columns', None)

In [7]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import geodesic
import requests

response = requests.get('https://api.data.gov.my/weather/forecast')
wfcast_json = response.json()
wfcast_df = pd.json_normalize(wfcast_json)

wfcast_df = wfcast_df[['date', 'summary_forecast', 'min_temp', 'max_temp', 'location.location_name']]
wfcast_df.rename(columns={'location.location_name': 'location_name'}, inplace=True)
points_df = pd.read_csv('weather_station_base.csv')

rain_table = wfcast_df.merge(points_df, on='location_name', how='left')
weather_gdf = gpd.GeoDataFrame(
    rain_table,
    geometry=gpd.points_from_xy(rain_table.Longitude, rain_table.Latitude),
    crs="EPSG:4326"
)
weather_gdf['date'] = pd.to_datetime(weather_gdf['date'])

earliest_date = weather_gdf['date'].min()

concessions = gpd.read_file("rspo_oil_palm/rspo_oil_palm_v20200114.shp").to_crs("EPSG:4326")
concessions = concessions[concessions['country'].isin(['Malaysia'])]
print(concessions.head())
print(len(concessions))

station_points = weather_gdf[['location_name', 'Longitude', 'Latitude']].drop_duplicates()
station_points['geometry'] = gpd.points_from_xy(station_points.Longitude, station_points.Latitude)
station_gdf = gpd.GeoDataFrame(station_points, geometry='geometry', crs='EPSG:4326')

# Find nearest station and calculate distance
def get_nearest_station_info(row):
    concession_centroid = row.geometry.centroid
    nearest_station = station_gdf.geometry.distance(concession_centroid).sort_values().index[0]
    nearest_row = station_gdf.loc[nearest_station]
    
    # Compute geodesic distance in km
    dist_km = geodesic(
        (concession_centroid.y, concession_centroid.x),
        (nearest_row.Latitude, nearest_row.Longitude)
    ).km
    
    return pd.Series({
        'nearest_station': nearest_row.location_name,
        'distance_km': round(dist_km, 2)
    })

# Apply to concessions
concessions[['nearest_station', 'distance_km']] = concessions.apply(get_nearest_station_info, axis=1)
print(len(concessions))

# Merge 7-day forecast from the matched station
concessions_forecast = pd.merge(
    concessions,
    weather_gdf,
    left_on='nearest_station',
    right_on='location_name',
    how='left'
)

concessions_forecast_comp = concessions_forecast.reset_index(drop=True)
print(len(concessions_forecast_comp))

    gfw_fid  objectid                       group_  \
57      126       126        Benta Wawasan Sdn Bhd   
78      127       127  Boustead Plantations Berhad   
79      168       171  Boustead Plantations Berhad   
88      144       145  Boustead Plantations Berhad   
94      145       146  Boustead Plantations Berhad   

                        company             plantation rspocert memberyear  \
57        Benta Wawasan Sdn Bhd  Benta Wawasan Sdn Bhd     True       None   
78  Boustead Plantations Berhad                  Balau    False       None   
79  Boustead Plantations Berhad        Sungai-Sungai I    False       None   
88  Boustead Plantations Berhad       Ketengah Perwira    False       None   
94  Boustead Plantations Berhad       Ketengah Perwira    False       None   

         state       province   country  shape_leng  shape_area    gfw_area__  \
57       Sabah           None  Malaysia    1.329437    0.040697  49932.508955   
78    Selangor           None  Malaysia    0

C:\Users\DELL\AppData\Local\Temp\ipykernel_25280\3160012653.py:38: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  nearest_station = station_gdf.geometry.distance(concession_centroid).sort_values().index[0]


617
5544


In [9]:
type(concessions_forecast_comp)

geopandas.geodataframe.GeoDataFrame

In [6]:
concessions_forecast_comp.head(3)

,gfw_fid,objectid,group_,company,plantation,rspocert,memberyear,state,province,country,shape_leng,shape_area,gfw_area__,gfw_geosto,created_on,updated_on,geometry_x,nearest_station,distance_km,date,summary_forecast,min_temp,max_temp,location_name,Latitude,Longitude,geometry_y
0,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.1,2025-06-24,Ribut petir di beberapa tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
1,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.1,2025-06-23,Ribut petir di beberapa tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)
2,126,126,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,Benta Wawasan Sdn Bhd,True,None,Sabah,None,Malaysia,1.329437,0.040697,49932.508955,1dd4322e-cb5a-f0fa-ecb3-f6c3ff71e4b2,2020-07-28,2020-07-28,"POLYGON ((117.65473 4.63209, 117.65472 4.63141...",Tawau,61.1,2025-06-22,Ribut petir di beberapa tempat,23,33,Tawau,4.2447,117.8912,POINT (117.8912 4.2447)


In [39]:
concessions_forecast_comp['date'] = pd.to_datetime(concessions_forecast_comp['date'])

def categorize_weather(weather):
    weather = weather.lower()
    if 'hujan' in weather and 'tiada' not in weather:
        return 'Hujan'
    elif 'ribut petir' in weather:
        return 'Ribut Petir'
    elif 'berangin' in weather:
        return 'Berangin'
    elif 'tiada hujan' in weather:
        return 'Tiada Hujan'
    return weather  # fallback

concessions_forecast_comp['summary_forecast'] = concessions_forecast_comp['summary_forecast'].apply(categorize_weather)
summary_df = (
    concessions_forecast_comp.groupby(['date', 'summary_forecast'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

In [40]:
summary_df

summary_forecast,date,Berangin,Hujan,Ribut Petir,Tiada Hujan
0,2025-06-14,0,226,452,114
1,2025-06-15,0,135,620,37
2,2025-06-16,0,0,775,17
3,2025-06-17,0,61,656,75
4,2025-06-18,0,123,622,47
5,2025-06-19,0,176,460,156
6,2025-06-20,5,103,448,236


In [ ]:
df = pd.read_csv('TradeData_6_15_2025_13_30_32.csv', encoding='latin1')

In [39]:
df1 = pd.read_csv('TradeData_6_15_2025_13_30_32.csv', encoding='latin1')
df2 = pd.read_csv('TradeData_6_15_2025_15_21_46.csv', encoding='latin1')
df3 = pd.read_csv('TradeData_6_15_2025_15_22_25.csv', encoding='latin1')

In [42]:
df3.columns

Index(['typeCode', 'freqCode', 'refPeriodId', 'refYear', 'refMonth', 'period',
       'reporterCode', 'reporterISO', 'reporterDesc', 'flowCode', 'flowDesc',
       'partnerCode', 'partnerISO', 'partnerDesc', 'partner2Code',
       'partner2ISO', 'partner2Desc', 'classificationCode',
       'classificationSearchCode', 'isOriginalClassification', 'cmdCode',
       'cmdDesc', 'aggrLevel', 'isLeaf', 'customsCode', 'customsDesc',
       'mosCode', 'motCode', 'motDesc', 'qtyUnitCode', 'qtyUnitAbbr', 'qty',
       'isQtyEstimated', 'altQtyUnitCode', 'altQtyUnitAbbr', 'altQty',
       'isAltQtyEstimated', 'netWgt', 'isNetWgtEstimated', 'grossWgt',
       'isGrossWgtEstimated', 'cifvalue', 'fobvalue', 'primaryValue',
       'legacyEstimationFlag', 'isReported', 'isAggregate'],
      dtype='object')

In [43]:
df_combined = pd.concat([df1, df2, df3], ignore_index=True)

In [37]:
dff = df[['reporterISO', 'partnerISO', 'reporterDesc', 'refMonth', 'cmdCode', 'fobvalue']]

In [38]:
dff

,reporterISO,partnerISO,reporterDesc,refMonth,cmdCode,fobvalue
C,Malaysia,World,M,2023,Palm oil and its fractions; whether or not ref...,8.376651e+08
C,Malaysia,China,M,2023,Palm oil and its fractions; whether or not ref...,2.377219e+04
C,Malaysia,Ecuador,M,2023,Palm oil and its fractions; whether or not ref...,1.409224e+06
C,Malaysia,Germany,M,2023,Palm oil and its fractions; whether or not ref...,4.566481e+03
C,Malaysia,Indonesia,M,2023,Palm oil and its fractions; whether or not ref...,7.824195e+08
...,...,...,...,...,...,...
C,Malaysia,Egypt,X,2023,Vegetable oils; palm kernel or babassu oil and...,8.168563e+06
C,Malaysia,United Kingdom,X,2023,Vegetable oils; palm kernel or babassu oil and...,3.277451e+05
C,Malaysia,United Rep. of Tanzania,X,2023,Vegetable oils; palm kernel or babassu oil and...,2.170208e+04
C,Malaysia,USA,X,2023,Vegetable oils; palm kernel or babassu oil and...,3.318611e+07


In [39]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

completion = client.chat.completions.create(
  model="meta-llama/llama-3.1-8b-instruct",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)

print(completion.choices[0].message.content)


The meaning of life is a question that has puzzled philosophers, theologians, scientists, and thinkers for centuries. It is a complex and multifaceted question that can be explored from various perspectives, and there is no one definitive answer. Here are some possible perspectives on the meaning of life:

**Philosophical perspectives:**

1. **Existentialism:** Existentialist philosophers like Jean-Paul Sartre and Martin Heidegger argue that life has no inherent meaning; it's up to each individual to create their own purpose and meaning.
2. **Hedonism:** Epicureanism and hedonism propose that the primary goal of life is to seek happiness, pleasure, and self-indulgence.
3. **Stoicism:** Stoics like Epictetus and Seneca advocate for living in accordance with reason and virtue, accepting the natural order of the universe.
4. **Nihilism:** Nihilist philosophers reject the idea that life has inherent meaning, arguing that life is inherently meaningless, and our existence is just a fleeting 